In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torchvision.transforms import v2
from datasets import load_dataset
import tiktoken
import time
import math

In [2]:
# taken from https://pytorch-tutorials-preview.netlify.app/beginner/transformer_tutorial.html

class PositionalEncoding(nn.Module):

    def __init__(self, 
                 d_model: int, 
                 dropout: float = 0.1, 
                 max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[batch_size, seq_len, embedding_dim]``
        """
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [3]:
class SelfAttention(nn.Module):
    def __init__(self, 
                 d_model: int, 
                 d_query: int = 128, 
                 n_heads: int = 8,
                 device: torch.device = torch.device("cpu")):
        super().__init__()
        self.device = device
        self.n_heads = n_heads

        self.W_q = nn.Linear(d_model, d_query)
        self.W_k = nn.Linear(d_model, d_query)
        self.W_v = nn.Linear(d_model, d_model)

        self.scaling_factor = math.sqrt(d_query)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        
        x = x.unsqueeze(1).repeat([1, self.n_heads, 1, 1])
        
        q = self.W_q(x)
        k = self.W_k(x)
        v = self.W_v(x)

        attention_pattern = torch.matmul(q, torch.transpose(k, -2, -1)) / self.scaling_factor
        
        seq_len = attention_pattern.shape[-1]
        mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool().to(self.device)
        attention_pattern = torch.masked_fill(attention_pattern, mask, float("-inf"))

        attention_pattern = self.softmax(attention_pattern)

        output = torch.sum(torch.matmul(attention_pattern, v), dim=1)
        
        return output



In [4]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, 
                 d_model: int, 
                 d_up: int = 256):
        super().__init__()

        self.up = nn.Linear(d_model, d_up)
        self.relu = nn.ReLU()
        self.down = nn.Linear(d_up, d_model)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:

        output = self.up(x)
        output = self.relu(output)
        output = self.down(output)

        output = output + x

        return output

In [5]:
class Transformer(nn.Module):
    def __init__(self, 
                 n_vocab: int, 
                 d_model: int = 128, 
                 d_query: int = 128, 
                 n_heads: int = 8, 
                 n_layers: int = 4, 
                 d_up: int = 256,
                 device: torch.device = torch.device("cpu")):
        super().__init__()

        self.embedding = nn.Embedding(n_vocab, d_model)
        self.pe = PositionalEncoding(d_model, max_len=50000)

        self.attention_layers = nn.ModuleList([layer for _ in range(n_layers) for layer in 
                                               (SelfAttention(d_model, d_query, n_heads, device), 
                                                nn.LayerNorm(d_model),
                                                MultilayerPerceptron(d_model, d_up))])

        # self.self_attention = SelfAttention(d_model, d_query, n_heads, device)
        # self.mlp = MultilayerPerceptron(d_model, d_up)

        self.unembedding = nn.Linear(d_model, n_vocab)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x)
        x = self.pe(x)

        for layer in self.attention_layers:
            x = layer(x)

        x = self.unembedding(x)
        
        return x

In [6]:
class ModelCheckpoint():
    def __init__(self, model_state, optim_state, epoch: int, batch: int, rng_state: torch.Tensor):
        self.model_state = model_state
        self.optim_state = optim_state
        self.epoch       = epoch
        self.batch       = batch
        self.rng_state   = rng_state
    def save(self, file_path):
        torch.save({
            "model_state": self.model_state,
            "optim_state": self.optim_state,
            "epoch"      : self.epoch,
            "batch"      : self.batch,
            "rng_state"  : self.rng_state
        }, file_path)
        print(f"Model checkpoint saved at {file_path} at epoch {self.epoch} batch {self.batch}")
    @staticmethod
    def load(file_path):
        checkpoint = torch.load(file_path)
        return ModelCheckpoint(checkpoint["model_state"], 
                               checkpoint["optim_state"], 
                               checkpoint["epoch"], 
                               checkpoint["batch"], 
                               checkpoint["rng_state"])

In [7]:
def validate_model(model, 
                   device, 
                   criterion, 
                   test_loader):
    with torch.no_grad():
        avg_loss = 0
        for idx, inputs in enumerate(test_loader):
            inputs = inputs.to(device)
            targets = inputs[:,1:]
            outputs = model(inputs)[:,:-1,:]

            targets = targets.reshape(-1)
            outputs = outputs.reshape(-1, outputs.shape[-1])
            
            loss = criterion(outputs, targets)

            avg_loss += loss

        avg_loss /= len(test_loader)
        print(f"VALIDATE: Average Loss: {avg_loss}")

def train_model(model: nn.Module, 
                optimizer, 
                criterion, 
                device, 
                train_loader, 
                validation_loader, 
                accum_steps,
                num_epochs: int = 4,
                checkpoint: ModelCheckpoint = None):
    
    epoch = 0
    start_batch = 0

    if checkpoint != None:
        model.load_state_dict(checkpoint.model_state)
        optimizer.load_state_dict(checkpoint.optim_state)
        epoch = checkpoint.epoch
        start_batch = checkpoint.batch
        torch.set_rng_state(checkpoint.rng_state)


    model.train()
    
    for epoch in range(epoch, num_epochs):
        start_time = time.time()
        for idx, inputs in enumerate(train_loader):
            if idx > start_batch:
                inputs = inputs.to(device)
                targets = inputs[:,1:]
                outputs = model(inputs)[:,:-1,:]

                targets = targets.reshape(-1)
                outputs = outputs.reshape(-1, outputs.shape[-1])
                
                loss = criterion(outputs, targets) / accum_steps
                loss.backward()

                if (idx + 1) % accum_steps == 0:
                    optimizer.step()
                    optimizer.zero_grad()

                if (idx + 1) % (accum_steps * 4) == 0:
                    print(f"Epoch [{epoch}].[{idx}] Loss: {loss * accum_steps}")

                if (idx + 1) % (accum_steps * 16) == 0:
                    elapsed_time = time.time() - start_time
                    print(f"TIME: {elapsed_time / (accum_steps * 16)} seconds per batch")
                    start_time = time.time()

                    allocated = torch.cuda.memory_allocated() / 1e9
                    reserved = torch.cuda.memory_reserved() / 1e9
                    peak = torch.cuda.max_memory_allocated() / 1e9
                    print(f"USAGE: Allocated {allocated:.2f}GB, Reserved {reserved:.2f}GB, Peak: {peak:.2f}GB")

                if (idx + 1) % (accum_steps * 32) == 0:
                    validate_model(model, device, criterion, validation_loader)
                    checkpoint = ModelCheckpoint(model.state_dict(), 
                                                optimizer.state_dict(), 
                                                epoch, 
                                                idx, 
                                                torch.get_rng_state())
                    checkpoint.save("checkpoint.pt")

            

In [8]:
torch.manual_seed(42)
torch.set_float32_matmul_precision('high')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [9]:
dataset = load_dataset("roneneldan/TinyStories", split="train+validation")
dataset

Dataset({
    features: ['text'],
    num_rows: 2141709
})

In [10]:
encoder = tiktoken.get_encoding("cl100k_base")

In [11]:
def tokenize(sequence):
    sequence["text"] = torch.tensor(encoder.encode(sequence["text"]), dtype=torch.int64)
    return sequence

tokenized_dataset = dataset.map(tokenize, num_proc=8).with_format("torch")
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2, shuffle=True)

In [12]:
train = tokenized_dataset["train"]["text"]

test_set = tokenized_dataset["test"].train_test_split(test_size=0.001, shuffle=True)
test = test_set["train"]["text"]
validation = test_set["test"]["text"]

In [13]:
# hyperparameters
batch_size = 2
accum_steps = 32
d_model  = 256
d_query  = 64
d_up = 512
n_heads  = 4
n_layers = 4

n_vocab  = encoder.n_vocab

In [14]:
print(n_vocab)

100277


In [15]:
test_tensor = torch.randn(2,2,2)
test_tensor = test_tensor.unsqueeze(1)
test_tensor = test_tensor.repeat(1,4,1,1)
print(test_tensor)
print(test_tensor.shape)
test_tensor = torch.sum(test_tensor, dim=1)
print(test_tensor)
print(test_tensor.shape)

tensor([[[[ 0.3367,  0.1288],
          [ 0.2345,  0.2303]],

         [[ 0.3367,  0.1288],
          [ 0.2345,  0.2303]],

         [[ 0.3367,  0.1288],
          [ 0.2345,  0.2303]],

         [[ 0.3367,  0.1288],
          [ 0.2345,  0.2303]]],


        [[[-1.1229, -0.1863],
          [ 2.2082, -0.6380]],

         [[-1.1229, -0.1863],
          [ 2.2082, -0.6380]],

         [[-1.1229, -0.1863],
          [ 2.2082, -0.6380]],

         [[-1.1229, -0.1863],
          [ 2.2082, -0.6380]]]])
torch.Size([2, 4, 2, 2])
tensor([[[ 1.3468,  0.5152],
         [ 0.9378,  0.9213]],

        [[-4.4914, -0.7453],
         [ 8.8328, -2.5520]]])
torch.Size([2, 2, 2])


In [16]:
def collate_fn_padding(batch):
    batch = pad_sequence(batch, batch_first=True)
    return batch

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn_padding)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn_padding)
validation_loader = DataLoader(validation, batch_size=batch_size, shuffle=True, collate_fn=collate_fn_padding)

In [17]:
model = Transformer(n_vocab=n_vocab, 
                    d_model=d_model, 
                    d_query=d_query, 
                    n_heads=n_heads, 
                    n_layers=n_layers, 
                    d_up=d_up, 
                    device=device).to(device)
# model = torch.compile(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# checkpoint = ModelCheckpoint.load("checkpoint.pt")

train_model(model=model, 
            optimizer=optimizer, 
            criterion=criterion, 
            device=device, 
            train_loader=train_loader, 
            validation_loader=validation_loader, 
            accum_steps=accum_steps)

Epoch [0].[127] Loss: 11.020341873168945
Epoch [0].[255] Loss: 10.065437316894531
Epoch [0].[383] Loss: 8.589371681213379
Epoch [0].[511] Loss: 8.965344429016113
TIME: 0.028048223815858364 seconds per batch
USAGE: Allocated 0.85GB, Reserved 4.78GB, Peak: 4.36GB
Epoch [0].[639] Loss: 9.545528411865234
Epoch [0].[767] Loss: 9.132296562194824
Epoch [0].[895] Loss: 7.2856364250183105
Epoch [0].[1023] Loss: 7.764287948608398
TIME: 0.026772727724164724 seconds per batch
USAGE: Allocated 0.88GB, Reserved 4.78GB, Peak: 4.36GB
VALIDATE: Average Loss: 8.018267631530762
Model checkpoint saved at checkpoint.pt at epoch 0 batch 1023
Epoch [0].[1151] Loss: 8.217638969421387
Epoch [0].[1279] Loss: 8.103324890136719
Epoch [0].[1407] Loss: 6.887085437774658
Epoch [0].[1535] Loss: 6.411445617675781
TIME: 0.03203374054282904 seconds per batch
USAGE: Allocated 0.88GB, Reserved 4.78GB, Peak: 4.36GB
Epoch [0].[1663] Loss: 5.6081037521362305
Epoch [0].[1791] Loss: 6.965433120727539
Epoch [0].[1919] Loss: 6.5

In [ ]:
def generate_response(model, encoder: tiktoken.Encoding, device: torch.device, prompt: str):
    sequence = encoder.encode(prompt)
    with torch.no_grad():
        for _ in range(100):
            input = torch.tensor(sequence, dtype=torch.int64).unsqueeze(0).to(device)
            output = model(input)
            output = output[0,-1,:].argmax().item()
            sequence = sequence + [output]

        response = encoder.decode(sequence)
        return response


In [ ]:
# checkpoint = ModelCheckpoint.load("checkpoint.pt")
# model.load_state_dict(checkpoint.model_state)

prompt = "Tell me a story about a hungry man."

response = generate_response(model, encoder=encoder, device=device, prompt=prompt)
print(response)

TorchRuntimeError: Dynamo failed to run FX node with fake tensors: call_function <built-in method matmul of type object at 0x79360d9dcaa0>(*(FakeTensor(..., device='cuda:0', size=(1, 4, 9, 64)), FakeTensor(..., device='cuda:0', size=(1, 9, 4, 64))), **{}): got RuntimeError('Attempting to broadcast a dimension of length 9 at -1! Mismatching argument at index 1 had [1, 9]; but expected shape should be broadcastable to [1, 4]')

from user code:
   File "/tmp/ipykernel_109531/47181234.py", line 30, in forward
    x = layer(x)
  File "/tmp/ipykernel_109531/2885984990.py", line 27, in forward
    attention_pattern = torch.matmul(q, torch.transpose(k, 1, 2)) / self.scaling_factor

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


In [ ]:
output = model(encoder.encode(""))